<font size="10">Decision Tree</font>

In [76]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import OneHotEncoder

#dataset = pd.read_excel('loan.xlsx')
#X = dataset.iloc[:, 0:13].values
#y = dataset.iloc[:, 13].values
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [77]:
loan_df=pd.read_excel("loan.xlsx")
loan_df_cat=loan_df[['Sex', 'Age', 'Time_at_address', 'Res_status', 'Telephone',
       'Occupation', 'Job_status', 'Time_employed', 'Time_bank', 'Liab_ref',
       'Acc_ref', 'Home_Expn', 'Balance']]
df_dt = pd.DataFrame(pd.get_dummies(loan_df_cat))
X = df_dt[['Home_Expn', 'Balance', 'Sex_F', 'Sex_M', 'Res_status_owner',
       'Res_status_rent', 'Telephone_given', 'Telephone_not_given',
       'Occupation_creative_', 'Occupation_driver', 'Occupation_executive',
       'Occupation_guard_etc', 'Occupation_labourer', 'Occupation_manager',
       'Occupation_office_st', 'Occupation_productio', 'Occupation_professio',
       'Occupation_sales', 'Occupation_semi_pro', 'Occupation_unemploye',
       'Job_status_governmen', 'Job_status_military', 'Job_status_private_s',
       'Job_status_retired', 'Job_status_self_empl', 'Job_status_student',
       'Job_status_unemploye', 'Liab_ref_f', 'Liab_ref_t', 'Acc_ref_given',
       'Acc_ref_oth_inst_', 'Age',
       'Time_employed', 'Time_bank']]

from sklearn.preprocessing import LabelEncoder
df_dt['Decision_label']= LabelEncoder().fit_transform(loan_df['Decision'])
y = df_dt['Decision_label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=101)

#var_train, var_test, res_train, res_test = train_test_split(header, result_list, test_size = 0.3)

from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

decision_tree = DecisionTreeClassifier()
decision_tree = decision_tree.fit(X_train, y_train)

y_pred = decision_tree.predict(X_test)
score = accuracy_score(y_test, y_pred)

print("Accuracy score of Decision Tree Algorithm = ", score)

Accuracy score of Decision Tree Algorithm =  0.6627906976744186


<font size="10">Random forest</font>

In [78]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=600)
rfc.fit(X_train,y_train)
predictions_RFC = rfc.predict(X_test)
score = accuracy_score(y_test, predictions_RFC)
print("Accuracy score of Random Forest Algorithm = ", score)

Accuracy score of Random Forest Algorithm =  0.7441860465116279


<font size="5">On 1-level dropping for Decision Tree and Random Forest</font>

In [79]:
X = X.drop('Occupation_driver', axis=1)
X = X.drop('Occupation_creative_', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=101)

decision_tree = decision_tree.fit(X_train, y_train)
y_pred = decision_tree.predict(X_test)
score = accuracy_score(y_test, y_pred)
print("Accuracy score of Decision Tree Algorithm = ", score)

rfc.fit(X_train,y_train)
predictions_RFC = rfc.predict(X_test)
score = accuracy_score(y_test, predictions_RFC)
print("Accuracy score of Random Forest Algorithm = ", score)

Accuracy score of Decision Tree Algorithm =  0.6744186046511628
Accuracy score of Random Forest Algorithm =  0.7558139534883721


<font size="3">We reeceived different accuracy, initially considered sets are better</font>

<font size="10">Decision Tree Diagram</font>

In [64]:
#imports

import numpy as np
import pandas as pd

df = pd.read_excel('loan.xlsx')
#print(df)

#dfcols = pd.DataFrame(columns = df.columns)
#print(dfcols)
#print(dfcols.columns)
#print(df.columns)

# list of row values
cols_list = df.to_numpy().tolist()
#print(cols_list)

In [65]:
# list of column headings
header = df.columns.values.tolist()
print(header)

['Sex', 'Age', 'Time_at_address', 'Res_status', 'Telephone', 'Occupation', 'Job_status', 'Time_employed', 'Time_bank', 'Liab_ref', 'Acc_ref', 'Home_Expn', 'Balance', 'Decision']


In [66]:
def unique_vals(rows, col):
    return set([row[col] for row in rows])

def class_counts(rows):
    counts = {} # a dictionary of label -> counts
    for row in rows:
        label = row[-1]    #last column
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

#-------
class Question:
    def __init__ (self, column, value):
        self.column = column
        self.value = value
        
    def match (self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__ (self):
        condition = "=="
        if is_numeric (self.value):
            condition = ">="
        return "Is %s %s %s?" % (header[self.column], condition, str(self.value))
#--------

def partition (rows, question):
    true_rows,false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append (row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1-p) * gini(right)

def find_best_split(rows):
    best_gain = 0
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1
    
    for col in range(n_features):
        values = set([ row[col] for row in rows])
        for val in values:
            question = Question(col, val)
            true_rows, false_rows = partition(rows, question)
            
            if (len(true_rows) == 0 or len(false_rows) ==0):
                continue
                
            gain = info_gain (true_rows, false_rows, current_uncertainty)
                
            if gain >= best_gain:
                best_gain, best_question = gain, question 
            
    return best_gain, best_question

#----
class Leaf:
    def __init__ (self, rows):
        self.predictions = class_counts(rows)

class Decision_Node:
    def __init__ (self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

def build_tree(rows):
    gain, question = find_best_split(rows)
    if gain == 0:
        return Leaf(rows)
    true_rows, false_rows = partition(rows, question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    return Decision_Node (question, true_branch, false_branch)

def print_tree( node, spacing=""):
    
    # base case, reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return
    
    #print question at this node
    print (spacing + str(node.question))
    
    #call this func requirsively on true branch
    print(spacing + '-->True:')
    print_tree (node.true_branch, spacing + " ")
    
    #call this func requirsively on false branch
    print(spacing + '-->False:')
    print_tree (node.false_branch, spacing + " ")

def classify (row, node):
    
    # base case, reached a leaf
    if isinstance (node, Leaf):
        return node.predictions
    
    #decide to follow true or false branch
    if node.question.match(row):
        return classify (row, node.true_branch)
    else:
        return classify (row, node.false_branch)

def print_leaf (counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] =  str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [67]:
if __name__ == '__main__':
    my_tree = build_tree (cols_list)
    print_tree (my_tree)
    
    testing_data = df.to_numpy().tolist()
    
    for row in testing_data:
        print ("Actual %s. Predicted: %s" % 
               (row[-1], print_leaf(classify(row, my_tree))))

Is Time_bank >= 3?
-->True:
 Is Time_employed >= 1?
 -->True:
  Is Time_bank >= 4?
  -->True:
   Is Time_employed >= 11?
   -->True:
    Is Time_employed >= 12?
    -->True:
     Predict {'accept': 8}
    -->False:
     Predict {'reject': 1}
   -->False:
    Predict {'accept': 62}
  -->False:
   Is Time_at_address >= 1.5?
   -->True:
    Is Occupation == productio?
    -->True:
     Is Balance >= 14?
     -->True:
      Predict {'accept': 1}
     -->False:
      Predict {'reject': 1}
    -->False:
     Predict {'accept': 9}
   -->False:
    Is Home_Expn >= 312?
    -->True:
     Predict {'accept': 1}
    -->False:
     Predict {'reject': 2}
 -->False:
  Is Age >= 36.6699981689453?
  -->True:
   Is Time_at_address >= 15.5?
   -->True:
    Predict {'reject': 1}
   -->False:
    Is Time_bank >= 5?
    -->True:
     Predict {'accept': 8}
    -->False:
     Is Home_Expn >= 180?
     -->True:
      Predict {'reject': 1}
     -->False:
      Predict {'accept': 1}
  -->False:
   Is Job_status 